In [0]:
import cv2, time, pandas 
# importing datetime class from datetime library 
from datetime import datetime 
# from google.colab import drive
# drive.mount('/content/drive')


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/NormalizedFace_v2.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
!unzip -q '/content/drive/My Drive/NormalizedFace_v2.zip'

In [0]:
!ls /content/NormalizedFace/ClientNormalized

0001		       0006_00_06_03_263.bmp  0011_01_07_03_319.bmp
0001_00_00_01_0.bmp    0006_00_06_03_264.bmp  0011_01_07_03_31.bmp
0001_00_00_01_101.bmp  0006_00_06_03_265.bmp  0011_01_07_03_320.bmp
0001_00_00_01_105.bmp  0006_00_06_03_266.bmp  0011_01_07_03_321.bmp
0001_00_00_01_109.bmp  0006_00_06_03_267.bmp  0011_01_07_03_322.bmp
0001_00_00_01_112.bmp  0006_00_06_03_268.bmp  0011_01_07_03_326.bmp
0001_00_00_01_116.bmp  0006_00_06_03_269.bmp  0011_01_07_03_327.bmp
0001_00_00_01_123.bmp  0006_00_06_03_26.bmp   0011_01_07_03_328.bmp
0001_00_00_01_127.bmp  0006_00_06_03_270.bmp  0011_01_07_03_329.bmp
0001_00_00_01_12.bmp   0006_00_06_03_271.bmp  0011_01_07_03_32.bmp
0001_00_00_01_130.bmp  0006_00_06_03_272.bmp  0011_01_07_03_330.bmp
0001_00_00_01_134.bmp  0006_00_06_03_273.bmp  0011_01_07_03_331.bmp
0001_00_00_01_138.bmp  0006_00_06_03_274.bmp  0011_01_07_03_332.bmp
0001_00_00_01_141.bmp  0006_00_06_03_275.bmp  0011_01_07_03_333.bmp
0001_00_00_01_145.bmp  0006_00_06_03_276.bmp  0011_01_07_03_

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import cv2

Using TensorFlow backend.


In [0]:
train_a = pd.read_csv("/content/NormalizedFace/client_train_normalized.txt", sep = "\t", header = None)
train_label = pd.read_csv("/content/NormalizedFace/imposter_train_normalized.txt", sep = "\t", header = None )

In [0]:
# train_a.head()

In [0]:
train_a["label"] = 0
train_label["label"] = 1

train = pd.concat([train_a, train_label], ignore_index= True)

In [0]:
train.rename(columns = {0:"id"}, inplace = True)

In [14]:
train.head()

,id,label
0,0001_00_00_01_0.bmp,0
1,0001_00_00_01_101.bmp,0
2,0001_00_00_01_105.bmp,0
3,0001_00_00_01_109.bmp,0
4,0001_00_00_01_112.bmp,0


In [120]:
# /content/NormalizedFace/ClientNormalized
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('/content/NormalizedFace/ImposterNormalized/'+train['id'][i], target_size=(224,224,1), color_mode="grayscale" )
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 9123/9123 [00:06<00:00, 1489.23it/s]


In [0]:
y=train['label'].values
trainY = to_categorical(y, num_classes=2)

In [0]:
# y = to_categorical(y)


In [0]:
# y = y.reshape(1, -1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [0]:
model = Sequential()
model.add(Conv2D(224, kernel_size=(3, 3),activation='relu',input_shape=(224,224,1)))
model.add(Conv2D(224, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(112, (3, 3), activation='relu'))
model.add(Conv2D(112, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(56, (3, 3), activation='relu'))
model.add(Conv2D(56, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(28, (3, 3), activation='relu'))
model.add(Conv2D(28, (3, 3), activation='relu'))
model.add(Conv2D(28, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(14, (3, 3), activation='relu'))
model.add(Conv2D(14, (3, 3), activation='relu'))
model.add(Conv2D(14, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(7, (3, 3), activation='relu'))
model.add(Conv2D(7, (3, 3), activation='relu'))
model.add(Conv2D(7, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.10))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])  

In [140]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 7298 samples, validate on 1825 samples
Epoch 1/10
7298/7298 [==============================] - 1139s 156ms/step - loss: 0.0950 - acc: 0.9697 - val_loss: 0.1718 - val_acc: 0.9622
Epoch 2/10
7298/7298 [==============================] - 1123s 154ms/step - loss: 0.0441 - acc: 0.9889 - val_loss: 0.1371 - val_acc: 0.9534
Epoch 3/10
7298/7298 [==============================] - 1117s 153ms/step - loss: 0.0190 - acc: 0.9948 - val_loss: 0.0170 - val_acc: 0.9945
Epoch 4/10
7298/7298 [==============================] - 1123s 154ms/step - loss: 0.1075 - acc: 0.9637 - val_loss: 0.0513 - val_acc: 0.9814
Epoch 5/10
7298/7298 [==============================] - 1123s 154ms/step - loss: 0.0414 - acc: 0.9866 - val_loss: 0.0584 - val_acc: 0.9912
Epoch 6/10
7298/7298 [==============================] - 1122s 154ms/step - loss: 0.0125 - acc: 0.9956 - val_loss: 0.1893 - val_acc: 0.9447
Epoch 7/10
7298/7298 [==============================] - 1119s 153ms/step - loss: 0.0265 - acc: 0.9926 - val_loss: 0.25

In [0]:
from keras.models import load_model
model.save('my_model.h5')

In [0]:
test_a = pd.read_csv("/content/NormalizedFace/client_test_normalized.txt", sep = "\t", header = None)
test_label = pd.read_csv("/content/NormalizedFace/imposter_test_normalized.txt", sep = "\t", header = None )

test_a["label"] = 0
test_label["label"] = 1

test = pd.concat([test_a, test_label], ignore_index= True)

test.rename(columns = {0: "id"}, inplace = True)

In [142]:
test.head()

,id,label
0,0010_01_05_03_115.bmp,0
1,0010_01_05_03_121.bmp,0
2,0010_01_05_03_125.bmp,0
3,0010_01_05_03_129.bmp,0
4,0010_01_05_03_131.bmp,0


In [145]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/content/NormalizedFace/ImposterNormalized/'+train['id'][i], target_size=(224,224,1), color_mode='grayscale')
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test_img = np.array(test_image)

100%|██████████| 9123/9123 [00:06<00:00, 1431.12it/s]


In [0]:
prediction = model.predict_classes(test_img)

In [0]:
from sklearn.metrics import accuracy_score

In [148]:
accuracy_score(test['label'], prediction.reshape(9123,))

0.9811465526690781